<a href="https://colab.research.google.com/github/gkswodnd6319/COVID19-Analysis/blob/main/covid19_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install yfinance
!pip3 install dash

     |████████████████████████████████| 5.5MB 4.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=62cc4f5fe0c87fd0712fa3a15026cd508e34442c301dea27f31c2e32902c8537
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 1.0MB 8.6MB/s 
     |████████████████████████████████| 3.5MB 26.2MB/s 
     |████████████████████████████████| 194kB 39.8MB/s 
     |████████████████████████████████| 1.8MB 45.4MB/s 
     |████████████████████████████████| 358kB 44.7MB/s 
  Created wheel for dash: filename=dash-1.17.0-cp36-none-any.whl size=83728 sha256=1c8497b4b0ac31f6779d683fd59516ec218c9b789be09beff5533fa080feff8d
  Stored in directory: /root/.cache/pip/wheels/e0/32/13/8

In [ ]:
import os
import requests
from pandas import json_normalize
import pandas as pdy

import yfinance as yf
import datetime
import numpy as np

import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8060 &')

### Get the public URL where you can access the Dash app. Copy this URL.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-11-09 01:03:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.236.74.205, 34.232.47.250, 34.206.161.174, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.236.74.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  51.9MB/s    in 0.3s    

2020-11-09 01:03:11 (51.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
class COVID19_Stock_Data():

    def __init__(self, ticker, interpolate_stock=True):
        self.stock_data, self.covid19_data, self.start_date = None, None, None
        self.stock_returns, self.covid19_returns = None, None
        self.correlation = None

        self.get_covid19_data(ticker=ticker)
        self.get_start_date(ticker=ticker)
        self.get_stock_data(ticker=ticker)

        if interpolate_stock: self.interpolate_stock_data()

        self.stock_returns = self.get_returns(self.stock_data)
        self.covid19_returns = self.get_returns(self.covid19_data)
        self.get_correlation()

    def get_stock_data(self, ticker, col_type='Close'):
        data = yf.Ticker(ticker)
        hist = data.history(start=self.start_date, auto_adjust=True)
        self.stock_data = hist[col_type].to_frame() if col_type else hist.copy()

    def get_covid19_data(self, ticker, col_type='NewCases'):
        country = yf.Ticker(ticker).info['country'].lower().replace(' ', '-')
        covid19_data = requests.get('https://api.covid19api.com/premium/country/' + country,
                                    headers={'X-Access-Token': '5cf9dfd5-3449-485e-b5ae-70a60e997864'})
        data = json_normalize(covid19_data.json())
        data = data[data.TotalCases != 0].reset_index(drop=True)
        data['Date'] = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
        data.set_index('Date', inplace=True)
        self.covid19_data = data[col_type].to_frame() if col_type else data.copy()

    def get_start_date(self, ticker):
        self.start_date = self.covid19_data.index[0]

    def get_returns(self, data):
        """
        param:
        data: pandas.DataFrame (with only 1 column)
        """
        col_name = data.columns[0]
        data[col_name + '_shifted'] = data[col_name].shift(-1)
        data.dropna(inplace=True)
        with np.errstate(divide='ignore'):
            data[col_name + '_returns'] = data.apply(
                lambda x: 0.0 if x[col_name] == 0 and x[col_name + '_shifted'] == 0 else (x[col_name + '_shifted'] - x[
                    col_name]) / x[col_name], axis=1)
        data = data[col_name + '_returns'].to_frame()
        return data

    def interpolate_stock_data(self):
        data = self.stock_data.copy()
        data_interpolated_dict = dict()
        data.sort_index(inplace=True)
        stock_dates = data.index
        for i in range(len(stock_dates) - 1):
            date_today, date_next = stock_dates[i], stock_dates[i + 1]
            stock_date_diff = (date_next - date_today).days
            if stock_date_diff > 1:
                daily_rate = (data['Close'][date_next] / data['Close'][date_today]) ** (1 / stock_date_diff)
                for j in range(1, stock_date_diff):
                    data_interpolated_dict[date_today + datetime.timedelta(j)] = data['Close'][date_today] * (
                                daily_rate ** j)
            else:
                continue
        data_interpolated = pd.DataFrame.from_dict(data_interpolated_dict, orient='index', columns=['Close'])
        df = pd.concat([data, data_interpolated]).sort_index()
        self.stock_data = df.copy()

    def get_correlation(self):
        data_merged = pd.merge(self.stock_returns, self.covid19_returns, left_index=True, right_index=True)
        data_merged = data_merged.replace([np.inf, -np.inf], np.nan).dropna()
        self.correlation = data_merged[data_merged.columns[0]].corr(data_merged[data_merged.columns[1]])

In [ ]:
%%writefile my_app1.py
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objects as go
from plotly.subplots import make_subplots

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Stock Analysis Tool')
    ],
        style={'width': '50%', 'display': 'inline-block'}
    ),
    html.Div([
        dcc.Input(
            id='ticker',
            placeholder='Enter a ticker...',
            type='text',
            value=''
        )
    ],
        style={'float': 'right', 'display': 'inline-block'}
    ),
    html.Div(id='correlation_text'),
    dcc.Graph(id='returns_graph')
], style={'width': '500'})


@app.callback(Output('returns_graph', 'figure'), [Input('ticker', 'value')])
def update_company(input_value):
    company = COVID19_Stock_Data(ticker=input_value.upper(), interpolate_stock=True)
    data = [
        go.Scatter(x=company.stock_data.index,
                   y=company.stock_data[company.stock_data.columns[0]],
                   name='Stock Price',
                   marker=dict(
                       color='rgb(55, 83, 109)'
                   ),
                   mode='lines+markers',
                   yaxis='y1'
                   ),
        go.Scatter(x=company.covid19_data.index,
                   y=company.covid19_data[company.covid19_data.columns[0]],
                   name='COVID-19 Daily Cases',
                   marker=dict(
                       color='rgb(26, 118, 255)'
                   ),
                   mode='lines+markers',
                   yaxis='y2'
                   )
    ]
    layout = go.Layout(
        title=input_value.upper() + " Stock Price and COVID-19 Daily Cases",
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=True,
        legend=dict(
            x=0.05,
            y=0.90
        ),
        margin=dict(l=40, r=0, t=40, b=30),
        xaxis1={
            'title': 'Date',
            'gridcolor': 'rgba(0, 0, 0, 0.1)'
        },
        yaxis1={
            'title': 'Stock Price',
            'gridcolor': 'rgba(55, 83, 109, 0.1)'
        },
        yaxis2={
            'title': 'Number of COVID-19 Daily Cases',
            'overlaying': 'y',
            'side': 'right',
            'gridcolor': 'rgba(26, 118, 255, 0.1)'
        }
    )

    figure = go.Figure(data=data, layout=layout)
    return figure


@app.callback(Output('correlation_text', 'children'), [Input('ticker', 'value')])
def update_correlation_text(input_value):
    company = COVID19_Stock_Data(ticker=input_value.upper(), interpolate_stock=True)
    return html.H3(
        'The correlation between {} stock price return and COVID-19 daily case rate is: {}%.'.format(
            input_value.upper(),
            round(company.correlation * 100, 4)
        )
    )



if __name__ == '__main__':
      app.run_server(debug = True,port = 8060)

Writing my_app1.py


Dash is running on http://127.0.0.1:8060/

 * Serving Flask app "my_app1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
